In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
cov_df_P0 = pd.read_csv("eval/results/coverage/P0/coverage_results_P0.csv")
cov_df_P1 = pd.read_csv("eval/results/coverage/P1/coverage_results_P1.csv")
cov_df_P2 = pd.read_csv("eval/results/coverage/P2/coverage_results_P2.csv")
cov_df_P3 = pd.read_csv("eval/results/coverage/P3/coverage_results_P3.csv")
cov_df_req = pd.read_csv("eval/results/coverage/requests-functions/coverage_results_requests_functions.csv")

In [ ]:
# Build paper-ready coverage table

coverage_summary = pd.DataFrame([
    {"strategy": "requests", **cov_df_req.iloc[0].to_dict()},
    {"strategy": "P0", **cov_df_P0.iloc[0].to_dict()},
    {"strategy": "P1", **cov_df_P1.iloc[0].to_dict()},
    {"strategy": "P2", **cov_df_P2.iloc[0].to_dict()},
    {"strategy": "P3", **cov_df_P3.iloc[0].to_dict()},
])

# Select only columns suitable for the paper
coverage_summary = coverage_summary[[
    "strategy",
    "tests_run",
    "percent_line_coverage",
    "percent_branch_coverage",
    "percent_total_coverage",
]].copy()

# Rename columns for LaTeX / paper readability
coverage_summary = coverage_summary.rename(columns={
    "strategy": "Strategy",
    "tests_run": "N tests",
    "percent_line_coverage": "Line coverage (\\%)",
    "percent_branch_coverage": "Branch coverage (\\%)",
    "percent_total_coverage": "Total coverage (\\%)",
})

# Round values for the paper
coverage_summary = coverage_summary.round(2)

coverage_summary



In [ ]:
# Print coverage table in the form required by the template

def df_to_latex_tabular(df: pd.DataFrame) -> str:
    cols = df.columns.tolist()
    col_format = "|l|" + "|".join(["c"] * (len(cols) - 1)) + "|"

    lines = []
    lines.append(f"\\begin{{tabular}}{{{col_format}}}")
    lines.append("\\hline")

    # Header
    header = " & ".join(cols) + " \\\\"
    lines.append(header)
    lines.append("\\hline")

    # Rows
    for _, row in df.iterrows():
        row_str = " & ".join(str(v) for v in row.values) + " \\\\"
        lines.append(row_str)
        lines.append("\\hline")

    lines.append("\\end{tabular}")
    return "\n".join(lines)


latex_tabular = df_to_latex_tabular(coverage_summary)
print(latex_tabular)



In [ ]:
import os

def save_latex(path: str, content: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        f.write(content)

#Save table txt in the repository
save_latex(
    "latex/tables/CLI/coverage_cli_improved.tex",
    latex_tabular
)

In [ ]:
import matplotlib.pyplot as plt

plot_df = coverage_summary.set_index("Strategy")[
    ["Line coverage (\\%)", "Branch coverage (\\%)", "Total coverage (\\%)"]
]

colors = ['deepskyblue', 'royalblue', 'midnightblue']

fig, ax = plt.subplots(figsize=(7, 4.5))

plot_df.plot(
    kind="bar",
    ax=ax,
    color=colors,
    width=0.7
)

ax.set_ylabel("Coverage (%)", fontsize=11, labelpad=12)
ax.set_xlabel("Prompt strategy", fontsize=11, labelpad=12)
ax.set_title("Coverage by prompt strategy", pad=25)
ax.tick_params(axis="both", which="major", labelsize=11)

# Proportional y-limit (no huge empty space)
y_max = plot_df.max().max()
ax.set_ylim(0, y_max * 1.20)  # 20% headroom

ax.legend(
    ["Line coverage", "Branch coverage", "Total coverage"],
    loc="upper center",
    bbox_to_anchor=(0.5, 1.05),
    ncol=3,
    frameon=False,
    fontsize=11,
    handlelength=1.5,
    columnspacing=1.2
)

ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

fig.subplots_adjust(top=0.82)

#Save image in the repository
plt.savefig(
    "latex/figures/CLI/coverage_cli_improved.png",
    dpi=300,
    bbox_inches="tight"
)

plt.show()
